In [58]:
# ===============================
# 1. IMPORTS AND SETUP
# ===============================

import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
from tqdm.auto import tqdm
import time
import warnings
warnings.filterwarnings('ignore')



In [ ]:
# ===============================
# 2. DATA DOWNLOAD AND SETUP
# ===============================

# Download dataset (run once)
!git clone https://github.com/spMohanty/PlantVillage-Dataset.git
data_dir = 'PlantVillage-Dataset/raw/color'



In [59]:
# ===============================
# 3. DATA PROCESSING FUNCTIONS
# ===============================

def define_paths(data_dir):
    """Veri setindeki tüm dosya yollarını ve etiketlerini toplar"""
    filepaths = []
    labels = []
    for fold in os.listdir(data_dir):
        foldpath = os.path.join(data_dir, fold)
        if os.path.isdir(foldpath):
            for file in os.listdir(foldpath):
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    filepaths.append(os.path.join(foldpath, file))
                    labels.append(fold)
    return pd.DataFrame({'filepaths': filepaths, 'labels': labels})

def split_df(df):
    """Veri setini train/val/test olarak böler"""
    train_df, dummy_df = train_test_split(df, train_size=0.8, stratify=df['labels'], random_state=42)
    val_df, test_df = train_test_split(dummy_df, train_size=0.5, stratify=dummy_df['labels'], random_state=42)
    return train_df.reset_index(drop=True), val_df.reset_index(drop=True), test_df.reset_index(drop=True)



In [60]:
# ===============================
# 4. DATASET CLASSES
# ===============================

class PlantMultiOutputDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform
        # Bitki türleri ve durumları
        self.plant_names = sorted(set(label.split("___")[0] for label in dataframe['labels']))
        self.status_names = sorted(set(label.split("___")[1] for label in dataframe['labels']))
        self.plant_map = {name: idx for idx, name in enumerate(self.plant_names)}
        self.status_map = {name.lower(): idx for idx, name in enumerate(self.status_names)}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row.filepaths).convert("RGB")
        plant_str, status_str = row.labels.split("___")
        plant_label = self.plant_map[plant_str]
        status_label = self.status_map[status_str.lower()]
        
        if self.transform:
            img = self.transform(image=np.array(img))['image']
        
        return img, torch.tensor(plant_label), torch.tensor(status_label)

class PlantOnlyDataset(Dataset):
    def __init__(self, dataframe, plant_map, transform=None):
        self.df = dataframe
        self.plant_map = plant_map
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row.filepaths).convert("RGB")
        plant_str = row.labels.split("___")[0]
        plant_label = self.plant_map[plant_str]
        
        if self.transform:
            img = self.transform(image=np.array(img))['image']
        
        return img, torch.tensor(plant_label)

class HealthOnlyDataset(Dataset):
    def __init__(self, dataframe, status_map, transform=None):
        self.df = dataframe
        self.status_map = status_map
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row.filepaths).convert("RGB")
        status_str = row.labels.split("___")[1]
        status_label = self.status_map[status_str.lower()]
        
        if self.transform:
            img = self.transform(image=np.array(img))['image']
        
        return img, torch.tensor(status_label)


In [61]:
# ===============================
# 5. MODEL ARCHITECTURES
# ===============================

class MultiOutputModel(nn.Module):
    def __init__(self, plant_output_dim, status_output_dim, dropout=0.5):
        super().__init__()
        self.backbone = models.resnet18(weights='IMAGENET1K_V1')
        self.backbone.fc = nn.Identity()
        # Dropout ve batch normalization ekleyelim (Overfitting önleme için dropout=0.5)
        self.dropout = nn.Dropout(dropout)
        self.bn = nn.BatchNorm1d(512)
        self.fc_plant = nn.Linear(512, plant_output_dim)
        self.fc_health = nn.Linear(512, status_output_dim)

    def forward(self, x):
        features = self.backbone(x)
        features = self.bn(features)
        features = self.dropout(features)
        return self.fc_plant(features), self.fc_health(features)

class SingleOutputModel(nn.Module):
    def __init__(self, output_dim, dropout=0.5):
        super().__init__()
        self.backbone = models.resnet18(weights='IMAGENET1K_V1')
        # Son katmanı değiştir (Overfitting önleme için dropout=0.5)
        self.backbone.fc = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(512, output_dim)
        )

    def forward(self, x):
        return self.backbone(x)



In [63]:
# ===============================
# 6. TRAINING FUNCTIONS
# ===============================

def train_epoch_multi(model, loader, optimizer, criterion, device, scheduler=None):
    model.train()
    total_loss = 0
    plant_correct = 0
    health_correct = 0
    total_samples = 0
    
    for x, y_plant, y_health in tqdm(loader, desc="Eğitim - Multi"):
        x, y_plant, y_health = x.to(device), y_plant.to(device), y_health.to(device)
        optimizer.zero_grad()
        
        out_plant, out_health = model(x)
        loss_plant = criterion(out_plant, y_plant)
        loss_health = criterion(out_health, y_health)
        loss = loss_plant + loss_health
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        plant_correct += (out_plant.argmax(1) == y_plant).sum().item()
        health_correct += (out_health.argmax(1) == y_health).sum().item()
        total_samples += y_plant.size(0)
    
    if scheduler:
        scheduler.step()
    
    avg_loss = total_loss / len(loader)
    plant_acc = plant_correct / total_samples
    health_acc = health_correct / total_samples
    
    return avg_loss, plant_acc, health_acc

def val_epoch_multi(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    plant_correct = 0
    health_correct = 0
    total_samples = 0
    
    with torch.no_grad():
        for x, y_plant, y_health in tqdm(loader, desc="Val - Multi"):
            x, y_plant, y_health = x.to(device), y_plant.to(device), y_health.to(device)
            
            out_plant, out_health = model(x)
            loss_plant = criterion(out_plant, y_plant)
            loss_health = criterion(out_health, y_health)
            loss = loss_plant + loss_health
            
            total_loss += loss.item()
            plant_correct += (out_plant.argmax(1) == y_plant).sum().item()
            health_correct += (out_health.argmax(1) == y_health).sum().item()
            total_samples += y_plant.size(0)
    
    avg_loss = total_loss / len(loader)
    plant_acc = plant_correct / total_samples
    health_acc = health_correct / total_samples
    
    return avg_loss, plant_acc, health_acc

def train_epoch_single(model, loader, optimizer, criterion, device, scheduler=None):
    model.train()
    total_loss = 0
    correct = 0
    total_samples = 0
    
    for x, y in tqdm(loader, desc="Eğitim - Single"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        
        out = model(x)
        loss = criterion(out, y)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        correct += (out.argmax(1) == y).sum().item()
        total_samples += y.size(0)
    
    if scheduler:
        scheduler.step()
    
    return total_loss / len(loader), correct / total_samples

def val_epoch_single(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total_samples = 0
    
    with torch.no_grad():
        for x, y in tqdm(loader, desc="Val - Single"):
            x, y = x.to(device), y.to(device)
            
            out = model(x)
            loss = criterion(out, y)
            
            total_loss += loss.item()
            correct += (out.argmax(1) == y).sum().item()
            total_samples += y.size(0)
    
    return total_loss / len(loader), correct / total_samples



In [64]:
# ===============================
# 7. MAIN EXPERIMENT FUNCTION
# ===============================

def run_experiment(data_dir, num_epochs=10, batch_size=32, learning_rate=1e-4):
    """Complete experiment pipeline"""
    import torch
    from pathlib import Path
    
    print("Plant Disease Classification Experiment")
    print("=" * 50)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")
    
    # Data preparation
    print("\nData preparation...")
    df = define_paths(data_dir)
    print(f"Total samples: {len(df)}")
    print(f"Number of classes: {df['labels'].nunique()}")
    
    train_df, val_df, test_df = split_df(df)
    print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")
    
    # Data transforms - Overfitting önleme için güçlendirilmiş augmentation
    train_transform = A.Compose([
        A.RandomResizedCrop(size=(224, 224), scale=(0.7, 1.0)),  # Daha agresif crop
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.3),  # Yeni: Dikey flip
        A.RandomRotate90(p=0.3),  # Yeni: 90 derece rotasyon
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.5),  # Yeni
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),  # Güçlendirildi
        A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1, p=0.5),  # Güçlendirildi
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),  # Yeni: Gürültü ekleme
        A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.3),  # Yeni: Random erasing
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])
    
    val_transform = A.Compose([
        A.Resize(224, 224),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])
    
    # Multi-output dataset
    multi_dataset_train = PlantMultiOutputDataset(train_df, transform=train_transform)
    multi_dataset_val = PlantMultiOutputDataset(val_df, transform=val_transform)
    
    plant_names = multi_dataset_train.plant_names
    status_names = multi_dataset_train.status_names
    plant_map = multi_dataset_train.plant_map
    status_map = multi_dataset_train.status_map
    
    print(f"Plant species: {len(plant_names)}")
    print(f"Health statuses: {len(status_names)}")
    
    # DataLoaders
    train_loader_multi = DataLoader(multi_dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader_multi = DataLoader(multi_dataset_val, batch_size=batch_size, shuffle=False, num_workers=0)
    
    plant_only_train = PlantOnlyDataset(train_df, plant_map, transform=train_transform)
    plant_only_val = PlantOnlyDataset(val_df, plant_map, transform=val_transform)
    health_only_train = HealthOnlyDataset(train_df, status_map, transform=train_transform)
    health_only_val = HealthOnlyDataset(val_df, status_map, transform=val_transform)
    
    train_loader_plant = DataLoader(plant_only_train, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader_plant = DataLoader(plant_only_val, batch_size=batch_size, shuffle=False, num_workers=0)
    train_loader_health = DataLoader(health_only_train, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader_health = DataLoader(health_only_val, batch_size=batch_size, shuffle=False, num_workers=0)
    
    criterion = nn.CrossEntropyLoss()
    results = {}
    
    # =====================================
    # 1. MULTI-OUTPUT MODEL TRAINING
    # =====================================
    print("\n" + "="*30)
    print("MULTI-OUTPUT MODEL")
    print("="*30)
    
    model_multi = MultiOutputModel(len(plant_names), len(status_names)).to(device)
    # Overfitting önleme: Weight decay (L2 regularization) eklendi
    optimizer_multi = torch.optim.Adam(
        model_multi.parameters(), 
        lr=learning_rate,
        weight_decay=1e-4  # L2 Regularization - Overfitting önler
    )
    # Learning rate scheduler güçlendirildi
    scheduler_multi = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer_multi, 
        mode='min', 
        factor=0.5,  # Learning rate'i yarıya indir
        patience=3,  # 3 epoch beklenmeden azalt
        min_lr=1e-6  # Minimum learning rate
    )
    
    train_losses_multi = []
    val_losses_multi = []
    val_plant_accs = []
    val_health_accs = []
    
    # Early Stopping - Overfitting önleme
    class EarlyStopping:
        def __init__(self, patience=5, min_delta=0.001):
            self.patience = patience
            self.min_delta = min_delta
            self.counter = 0
            self.best_loss = float('inf')
            
        def __call__(self, val_loss):
            if val_loss < self.best_loss - self.min_delta:
                self.best_loss = val_loss
                self.counter = 0
                return False  # Devam et
            else:
                self.counter += 1
                if self.counter >= self.patience:
                    return True  # Dur
                return False  # Devam et
    
    early_stopping = EarlyStopping(patience=5, min_delta=0.001)
    
    start_time = time.time()
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        
        train_loss, train_plant_acc, train_health_acc = train_epoch_multi(
            model_multi, train_loader_multi, optimizer_multi, criterion, device, None  # Scheduler'ı manuel çağıracağız
        )
        
        val_loss, val_plant_acc, val_health_acc = val_epoch_multi(
            model_multi, val_loader_multi, criterion, device
        )
        
        # Learning rate scheduler (ReduceLROnPlateau validation loss'a göre çalışır)
        scheduler_multi.step(val_loss)
        
        train_losses_multi.append(train_loss)
        val_losses_multi.append(val_loss)
        val_plant_accs.append(val_plant_acc)
        val_health_accs.append(val_health_acc)
        
        print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
        print(f"Train Plant: {train_plant_acc:.4f} | Train Health: {train_health_acc:.4f}")
        print(f"Val Plant: {val_plant_acc:.4f} | Val Health: {val_health_acc:.4f}")
        
        # Early stopping kontrolü
        if early_stopping(val_loss):
            print(f"\n⏹️  Early stopping at epoch {epoch+1}")
            print(f"   Best validation loss: {early_stopping.best_loss:.4f}")
            break
    
    multi_time = time.time() - start_time
    
    results['multi'] = {
        'final_plant_acc': val_plant_acc,
        'final_health_acc': val_health_acc,
        'train_losses': train_losses_multi,
        'val_losses': val_losses_multi,
        'val_plant_accs': val_plant_accs,
        'val_health_accs': val_health_accs,
        'training_time': multi_time
    }
    
    print(f"\nMulti-output Model Final Results:")
    print(f"Plant Accuracy: {val_plant_acc:.4f}")
    print(f"Health Accuracy: {val_health_acc:.4f}")
    print(f"Average Accuracy: {(val_plant_acc + val_health_acc)/2:.4f}")
    print(f"Training Time: {multi_time:.1f}s")
    
    # =====================================
    # 2. PLANT-ONLY MODEL TRAINING
    # =====================================
    print("\n" + "="*30)
    print("PLANT-ONLY MODEL")
    print("="*30)
    
    plant_model = SingleOutputModel(len(plant_names)).to(device)
    # Overfitting önleme: Weight decay eklendi
    optimizer_plant = torch.optim.Adam(
        plant_model.parameters(), 
        lr=learning_rate,
        weight_decay=1e-4
    )
    scheduler_plant = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer_plant, 
        mode='min', 
        factor=0.5, 
        patience=3, 
        min_lr=1e-6
    )
    
    train_losses_plant = []
    val_losses_plant = []
    val_accs_plant = []
    
    start_time = time.time()
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        
        train_loss, train_acc = train_epoch_single(
            plant_model, train_loader_plant, optimizer_plant, criterion, device, None
        )
        
        val_loss, val_acc = val_epoch_single(
            plant_model, val_loader_plant, criterion, device
        )
        
        scheduler_plant.step(val_loss)
        
        train_losses_plant.append(train_loss)
        val_losses_plant.append(val_loss)
        val_accs_plant.append(val_acc)
        
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
    
    plant_time = time.time() - start_time
    
    results['plant'] = {
        'final_accuracy': val_acc,
        'train_losses': train_losses_plant,
        'val_losses': val_losses_plant,
        'val_accs': val_accs_plant,
        'training_time': plant_time
    }
    
    print(f"\nPlant-only Model Final Results:")
    print(f"Accuracy: {val_acc:.4f}")
    print(f"Training Time: {plant_time:.1f}s")
    
    # =====================================
    # 3. HEALTH-ONLY MODEL TRAINING
    # =====================================
    print("\n" + "="*30)
    print("HEALTH-ONLY MODEL")
    print("="*30)
    
    health_model = SingleOutputModel(len(status_names)).to(device)
    # Overfitting önleme: Weight decay eklendi
    optimizer_health = torch.optim.Adam(
        health_model.parameters(), 
        lr=learning_rate,
        weight_decay=1e-4
    )
    scheduler_health = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer_health, 
        mode='min', 
        factor=0.5, 
        patience=3, 
        min_lr=1e-6
    )
    
    train_losses_health = []
    val_losses_health = []
    val_accs_health = []
    
    start_time = time.time()
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        
        train_loss, train_acc = train_epoch_single(
            health_model, train_loader_health, optimizer_health, criterion, device, None
        )
        
        val_loss, val_acc = val_epoch_single(
            health_model, val_loader_health, criterion, device
        )
        
        scheduler_health.step(val_loss)
        
        train_losses_health.append(train_loss)
        val_losses_health.append(val_loss)
        val_accs_health.append(val_acc)
        
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
    
    health_time = time.time() - start_time
    
    results['health'] = {
        'final_accuracy': val_acc,
        'train_losses': train_losses_health,
        'val_losses': val_losses_health,
        'val_accs': val_accs_health,
        'training_time': health_time
    }
    
    print(f"\nHealth-only Model Final Results:")
    print(f"Accuracy: {val_acc:.4f}")
    print(f"Training Time: {health_time:.1f}s")
    
    # =====================================
    # 4. FINAL COMPARISON
    # =====================================
    print("\n" + "="*50)
    print("FINAL COMPARISON")
    print("="*50)
    
    multi_avg = (results['multi']['final_plant_acc'] + results['multi']['final_health_acc']) / 2
    single_avg = (results['plant']['final_accuracy'] + results['health']['final_accuracy']) / 2
    
    print(f"\nMULTI-OUTPUT MODEL:")
    print(f"  Plant Accuracy: {results['multi']['final_plant_acc']:.4f}")
    print(f"  Health Accuracy: {results['multi']['final_health_acc']:.4f}")
    print(f"  Average Accuracy: {multi_avg:.4f}")
    print(f"  Training Time: {results['multi']['training_time']:.1f}s")
    
    print(f"\nSINGLE-OUTPUT MODELS:")
    print(f"  Plant Accuracy: {results['plant']['final_accuracy']:.4f}")
    print(f"  Health Accuracy: {results['health']['final_accuracy']:.4f}")
    print(f"  Average Accuracy: {single_avg:.4f}")
    print(f"  Combined Training Time: {results['plant']['training_time'] + results['health']['training_time']:.1f}s")
    
    print(f"\nPERFORMANCE DIFFERENCE:")
    plant_diff = results['multi']['final_plant_acc'] - results['plant']['final_accuracy']
    health_diff = results['multi']['final_health_acc'] - results['health']['final_accuracy']
    avg_diff = multi_avg - single_avg
    time_saving = (results['plant']['training_time'] + results['health']['training_time']) - results['multi']['training_time']
    
    print(f"  Plant Classification: {plant_diff:+.4f}")
    print(f"  Health Classification: {health_diff:+.4f}")
    print(f"  Average: {avg_diff:+.4f}")
    print(f"  Time Saved: {time_saving:.1f}s ({time_saving/(results['plant']['training_time'] + results['health']['training_time'])*100:.1f}%)")
    
    if avg_diff > 0:
        print(f"\nMulti-output model performs {avg_diff:.4f} points better on average!")
    else:
        print(f"\nSingle-output models perform {abs(avg_diff):.4f} points better on average!")
    
    # =====================================
    # OTOMATIK MODEL KAYDETME
    # =====================================
    print("\n" + "="*50)
    print("💾 MODEL KAYDEDİLİYOR...")
    print("="*50)
    
    bundle = {
        "state_dict": model_multi.state_dict(),
        "plant_names": plant_names,
        "status_names": status_names,
        "plant_output_dim": len(plant_names),
        "status_output_dim": len(status_names),
        "img_size": 224,
        "mean": [0.485, 0.456, 0.406],
        "std": [0.229, 0.224, 0.225],
    }
    
    out_path = Path("../backend/models/plantvillage_multi.pt")
    out_path.parent.mkdir(parents=True, exist_ok=True)
    
    # Eski modeli yedekle (eğer varsa)
    backup_path = Path("../backend/models/plantvillage_multi_backup.pt")
    if out_path.exists():
        import shutil
        shutil.copy2(out_path, backup_path)
        print(f"📦 Eski model yedeklendi: {backup_path}")
    
    torch.save(bundle, out_path)
    
    print(f"✅ Model başarıyla kaydedildi: {out_path}")
    print(f"   Plant classes: {len(plant_names)}")
    print(f"   Status classes: {len(status_names)}")
    print(f"   Model size: {out_path.stat().st_size / (1024*1024):.2f} MB")
    print("="*50)
    
    return results, model_multi, plant_names, status_names


In [ ]:
# Modeli kaydet
from pathlib import Path

bundle = {
    "state_dict": model_multi.state_dict(),
    "plant_names": plant_names,
    "status_names": status_names,
    "plant_output_dim": len(plant_names),
    "status_output_dim": len(status_names),
    "img_size": 224,
    "mean": [0.485, 0.456, 0.406],
    "std": [0.229, 0.224, 0.225],
}

out_path = Path("backend/models/plantvillage_multi.pt")
out_path.parent.mkdir(parents=True, exist_ok=True)
torch.save(bundle, out_path)

print(f"✅ Model başarıyla kaydedildi: {out_path.absolute()}")
print(f"   Plant classes: {len(plant_names)}")
print(f"   Status classes: {len(status_names)}")
print(f"   Model size: {out_path.stat().st_size / (1024*1024):.2f} MB")

In [ ]:
# Model kontrolü
import torch
from pathlib import Path

model_path = Path("backend/models/plantvillage_multi.pt")

print("🔍 Model Kontrolü")
print("="*50)

if not model_path.exists():
    print("❌ Dosya bulunamadı!")
else:
    print(f"✅ Dosya mevcut: {model_path}")
    print(f"   Boyut: {model_path.stat().st_size / (1024*1024):.2f} MB\n")
    
    bundle = torch.load(model_path, map_location="cpu")
    
    print("📦 İçerik:")
    print("-"*50)
    
    # Gerekli alanlar
    required = ["state_dict", "plant_names", "status_names"]
    all_ok = True
    
    for key in required:
        if key in bundle:
            val = bundle[key]
            if key == "state_dict":
                print(f"✅ {key}: {len(val)} katman")
            else:
                print(f"✅ {key}: {len(val)} öğe")
                if len(val) > 0:
                    print(f"   İlk 3: {val[:3]}")
        else:
            print(f"❌ {key}: EKSIK!")
            all_ok = False
    
    # Opsiyonel alanlar
    print("\n📊 Ek Bilgiler:")
    for key in ["plant_output_dim", "status_output_dim", "img_size", "mean", "std"]:
        if key in bundle:
            print(f"✅ {key}: {bundle[key]}")
    
    print("\n" + "="*50)
    if all_ok:
        print("✅ MODEL DOĞRU KAYDEDİLMİŞ!")
    else:
        print("❌ MODELDE SORUN VAR!")
    print("="*50)

In [65]:
# ===============================
# 8. RUN EXPERIMENT
# ===============================

# Run the complete experiment
# Model otomatik kaydedilecek (Cell 6'daki fonksiyon içinde)
data_dir = 'PlantVillage-Dataset/raw/color'
results, model_multi, plant_names, status_names = run_experiment(data_dir, num_epochs=10)

# Print learning curves summary
print("\n" + "="*50)
print("LEARNING CURVES SUMMARY")
print("="*50)

print("\nMulti-output Model - Plant Accuracy per Epoch:")
for i, acc in enumerate(results['multi']['val_plant_accs']):
    print(f"Epoch {i+1}: {acc:.4f}")

print("\nMulti-output Model - Health Accuracy per Epoch:")
for i, acc in enumerate(results['multi']['val_health_accs']):
    print(f"Epoch {i+1}: {acc:.4f}")

print("\nPlant-only Model - Accuracy per Epoch:")
for i, acc in enumerate(results['plant']['val_accs']):
    print(f"Epoch {i+1}: {acc:.4f}")

print("\nHealth-only Model - Accuracy per Epoch:")
for i, acc in enumerate(results['health']['val_accs']):
    print(f"Epoch {i+1}: {acc:.4f}")

Plant Disease Classification Experiment
Device: cpu

Data preparation...
Total samples: 54305
Number of classes: 38
Train: 43444, Val: 5430, Test: 5431
Plant species: 14
Health statuses: 21

MULTI-OUTPUT MODEL

Epoch 1/10


Val - Multi: 100%|██████████| 170/170 [03:33<00:00,  1.26s/it]


Train Loss: 1.1100 | Val Loss: 0.1006
Train Plant: 0.8788 | Train Health: 0.8035
Val Plant: 0.9941 | Val Health: 0.9801

Epoch 2/10


Val - Multi: 100%|██████████| 170/170 [03:31<00:00,  1.24s/it]


Train Loss: 0.4917 | Val Loss: 0.0821
Train Plant: 0.9487 | Train Health: 0.9030
Val Plant: 0.9965 | Val Health: 0.9786

Epoch 3/10


...

Train Loss: 0.1649 | Train Acc: 0.9457
Val Loss: 0.0268 | Val Acc: 0.9923

Epoch 9/10


Val - Single: 100%|██████████| 170/170 [03:26<00:00,  1.22s/it]


Train Loss: 0.1529 | Train Acc: 0.9506
Val Loss: 0.0271 | Val Acc: 0.9910

Epoch 10/10


Val - Single: 100%|██████████| 170/170 [03:24<00:00,  1.20s/it]


Train Loss: 0.1153 | Train Acc: 0.9634
Val Loss: 0.0154 | Val Acc: 0.9956

Health-only Model Final Results:
Accuracy: 0.9956
Training Time: 47136.1s

FINAL COMPARISON

MULTI-OUTPUT MODEL:
  Plant Accuracy: 0.9982
  Health Accuracy: 0.9915
  Average Accuracy: 0.9948
  Training Time: 41391.6s

SINGLE-OUTPUT MODELS:
  Plant Accuracy: 0.9982
  Health Accuracy: 0.9956
  Average Accuracy: 0.9969
  Combined Training Time: 87040.4s

PERFORMANCE DIFFERENCE:
  Plant Classification: +0.0000
  Health Classification: -0.0041
  Average: -0.0020
  Time Saved: 45648.8s (52.4%)

Single-output models perform 0.0020 points better on average!

💾 MODEL KAYDEDİLİYOR...
📦 Eski model yedeklendi: ../backend/models/plantvillage_multi_backup.pt
✅ Model başarıyla kaydedildi: ../backend/models/plantvillage_multi.pt
   Plant classes: 14
   Status classes: 21
   Model size: 42.79 MB

LEARNING CURVES SUMMARY

Multi-output Model - Plant Accuracy per Epoch:
Epoch 1: 0.9941
Epoch 2: 0.9965
Epoch 3: 0.9956
Epoch 4: 0.997